# A Tiny Temperature Converter Library
## Written in Julia (a naive approach)

Based on a wonderful example from _Erik Engheim_

We start with some type definitions!

In [1]:
abstract type Temperature end

struct Celsius <: Temperature
    value::Float64
end

struct Kelvin <: Temperature
   value::Float64 
end

struct Fahrenheit <: Temperature
    value::Float64
end

Define the promotion rules, so Julia knows our preferences when mixing types.

In [2]:
import Base: promote_rule  # we import the `promote_rule` function to add our own methods

promote_rule(::Type{Kelvin}, ::Type{Celsius})     = Kelvin
promote_rule(::Type{Fahrenheit}, ::Type{Celsius}) = Celsius
promote_rule(::Type{Fahrenheit}, ::Type{Kelvin})  = Kelvin

promote_rule (generic function with 127 methods)

We implement the conversion logic by adding methods to the `convert` function, available in the `Base` of Julia.

In [3]:
import Base: convert  # again, we add our own methods to `convert`

convert(::Type{Kelvin},  t::Celsius)     = Kelvin(t.value + 273.15)
convert(::Type{Kelvin},  t::Fahrenheit)  = Kelvin(Celsius(t))
convert(::Type{Celsius}, t::Kelvin)      = Celsius(t.value - 273.15)
convert(::Type{Celsius}, t::Fahrenheit)  = Celsius((t.value - 32)*5/9)
convert(::Type{Fahrenheit}, t::Celsius)  = Fahrenheit(t.value*9/5 + 32)
convert(::Type{Fahrenheit}, t::Kelvin)   = Fahrenheit(Celsius(t))

convert (generic function with 202 methods)

We can add some nice constructors, so we can initialise each 

In [4]:
Kelvin(t::Temperature) = convert(Kelvin, t)
Celsius(t::Temperature) = convert(Celsius, t)
Fahrenheit(t::Temperature) = convert(Fahrenheit, t)

Fahrenheit

In [5]:
Kelvin(Fahrenheit(23))

Kelvin(268.15)

In [6]:
Fahrenheit(3) + Celsius(4) + Kelvin(4)

LoadError: MethodError: no method matching +(::Fahrenheit, ::Celsius)
[0mClosest candidates are:
[0m  +(::Any, ::Any, [91m::Any[39m, [91m::Any...[39m) at /Applications/Julia-1.7.app/Contents/Resources/julia/share/julia/base/operators.jl:655

Adding arithmetic operations for our types by extending the `Base` operators.

In [7]:
import Base: +, -, *  # operators are functions, we can extend them easily!

+(x::Temperature, y::Temperature) = +(promote(x,y)...)
-(x::Temperature, y::Temperature) = -(promote(x,y)...)

+(x::T, y::T) where {T <: Temperature} = T(x.value + y.value)
-(x::T, y::T) where {T <: Temperature} = T(x.value - y.value)

*(x::Number, y::T) where {T <: Temperature} = T(x * y.value);

### Let's test it:

In [8]:
Celsius(37) + Kelvin(10)

Kelvin(320.15)

In [9]:
Celsius(Celsius(37) + Kelvin(10))

Celsius(47.0)

In [10]:
Fahrenheit(3) + Celsius(4) + Kelvin(4)

Kelvin(265.0388888888889)

### A nice way to create some syntactic sugar:

In [11]:
const °C = Celsius(1)
const °F = Fahrenheit(1)
const K = Kelvin(1);


In [12]:
5°F, 23K, 42°C

(Fahrenheit(5.0), Kelvin(23.0), Celsius(42.0))

In [13]:
42°C + 10K

Kelvin(325.15)

### Alright, let's have a look behind the scenes...

In [14]:
example_calculation() = Fahrenheit(2) + Celsius(3) + Kelvin(4)

example_calculation (generic function with 1 method)

In [15]:
@code_llvm example_calculation()

;  @ In[14]:1 within `example_calculation`
define [1 x double] @julia_example_calculation_1759() #0 {
top:
  ret [1 x double] [double 0x407077BBBBBBBBBB]
}


#### LLVM figured out that the function returns always the same value... Dooh ;)

### This is the "machine code":

In [16]:
@code_native example_calculation()

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[14]:1 within `example_calculation`
	mov	x8, #-4919131752989213765
	movk	x8, #30651, lsl #32
	movk	x8, #16496, lsl #48
	fmov	d0, x8
	ret
; └


### Now a function which is not constant ;)

In [17]:
another_calculation(a, b, c) = Fahrenheit(a) + Celsius(b) + Kelvin(c)

another_calculation (generic function with 1 method)

In [24]:
@code_llvm debuginfo=:none another_calculation(1, 2, 3)

define [1 x double] @julia_another_calculation_2047(i64 signext %0, i64 signext %1, i64 signext %2) #0 {
top:
  %3 = sitofp i64 %0 to double
  %4 = sitofp i64 %1 to double
  %5 = sitofp i64 %2 to double
  %6 = fadd double %3, -3.200000e+01
  %7 = fmul double %6, 5.000000e+00
  %8 = fdiv double %7, 9.000000e+00
  %9 = fadd double %8, %4
  %10 = fadd double %9, 2.731500e+02
  %11 = fadd double %10, %5
  %.fca.0.insert = insertvalue [1 x double] zeroinitializer, double %11, 0
  ret [1 x double] %.fca.0.insert
}


### And the machine code:

In [26]:
@code_native debuginfo=:none another_calculation(2.0, 3.0, 4.0)

	.section	__TEXT,__text,regular,pure_instructions
	mov	x8, #-4593671619917905920
	fmov	d3, x8
	fadd	d0, d0, d3
	fmov	d3, #5.00000000
	fmul	d0, d0, d3
	fmov	d3, #9.00000000
	fdiv	d0, d0, d3
	fadd	d0, d0, d1
	mov	x8, #7378697629483820646
	movk	x8, #4710, lsl #32
	movk	x8, #16497, lsl #48
	fmov	d1, x8
	fadd	d0, d0, d1
	fadd	d0, d0, d2
	ret


## What about the actual performance?

In [28]:
using BenchmarkTools

In [30]:
@benchmark another_calculation(temperatures...) setup=(temperatures=rand(3))

BenchmarkTools.Trial: 10000 samples with 985 evaluations.
 Range (min … max):  54.949 ns … 840.398 ns  ┊ GC (min … max): 0.00% … 93.07%
 Time  (median):     56.049 ns               ┊ GC (median):    0.00%
 Time  (mean ± σ):   56.926 ns ±  27.318 ns  ┊ GC (mean ± σ):  1.73% ±  3.35%

  ▅█▇▇▅▅▃▁▁ ▂▁▃▃ ▂▅██▅▅▃▃▂   ▁ ▂▂                 ▁▂▁▂ ▁▁      ▂
  █████████▇████▇██████████████████▆█▆██▇▇▇▆▆▇▇▅███████████▇█▄ █
  54.9 ns       Histogram: log(frequency) by time        59 ns <

 Memory estimate: 64 bytes, allocs estimate: 4.

### The power of multiple dispatch

We can easily define our own `rand` method for `Kelvin` temperatures ()

In [31]:
using Random
Random.rand(rng::AbstractRNG, ::Random.SamplerType{Kelvin}) =  Kelvin(rand() * 5000)

In [32]:
rand(Kelvin)

Kelvin(2122.5149017918416)

In [33]:
temperatures = rand(Kelvin, 1_000_000)

1000000-element Vector{Kelvin}:
 Kelvin(309.9522638246643)
 Kelvin(2239.3143507263367)
 Kelvin(4809.8505448873)
 Kelvin(4517.649046332409)
 Kelvin(2666.4756334193344)
 Kelvin(3699.234192861852)
 Kelvin(2106.5130732706557)
 Kelvin(3134.717316507572)
 Kelvin(4690.364425816824)
 Kelvin(2344.6427028808885)
 Kelvin(4508.480597356379)
 Kelvin(1706.5356245813434)
 Kelvin(4913.625178622309)
 ⋮
 Kelvin(4321.473912553685)
 Kelvin(1689.3772594519762)
 Kelvin(3034.348738916759)
 Kelvin(120.81726465502595)
 Kelvin(2673.3944327410236)
 Kelvin(1434.1698134085202)
 Kelvin(1948.3863151882886)
 Kelvin(194.9704616571818)
 Kelvin(3792.944877030177)
 Kelvin(685.8860842196545)
 Kelvin(2184.0380274481217)
 Kelvin(4561.10453542271)

### Is this fast?

In [34]:
@benchmark rand(Kelvin, 1_000_000)

BenchmarkTools.Trial: 2879 samples with 1 evaluation.
 Range (min … max):  1.090 ms …  23.342 ms  ┊ GC (min … max):  0.00% … 92.95%
 Time  (median):     1.525 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):   1.735 ms ± 606.374 μs  ┊ GC (mean ± σ):  11.68% ± 16.22%

             ▆█▇▅▃▂   ▁                    ▁▁▁▁▁  ▁▁▁▁▁       ▁
  █▃▁▁▁▁▁▁▁▃████████▇██▇▇▆▃▅▃▃▅▄▄▃▁▄▃▁▄▆▇███████████████▇█▇▅▆ █
  1.09 ms      Histogram: log(frequency) by time      3.05 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 2.

In [35]:
@benchmark rand(1_000_000)

BenchmarkTools.Trial: 4407 samples with 1 evaluation.
 Range (min … max):  666.708 μs …   3.097 ms  ┊ GC (min … max):  0.00% … 54.47%
 Time  (median):       1.002 ms               ┊ GC (median):     0.00%
 Time  (mean ± σ):     1.134 ms ± 304.320 μs  ┊ GC (mean ± σ):  11.42% ± 16.23%

              ▇█▇▆▆▅▄▄▄▂                  ▁▁▁  ▁▂▂▂▂▂▂▁▁▁       ▂
  ▇▄▁▁▁▁▁▁▁▁▁▆███████████▇▅▇▇▅▃▃▃▁▁▁▁▃▇▇▇████████████████████▆▇ █
  667 μs        Histogram: log(frequency) by time       2.02 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 2.

### Yep... 2ms, 2 allocs, 8 MiB memory

### Convert them to `Celsius` by using our type constructor and the `.`-notation for element-wise operation (similar to Matlab or `ufuncs` in numpy)

In [36]:
Celsius.(temperatures)

1000000-element Vector{Celsius}:
 Celsius(36.80226382466435)
 Celsius(1966.1643507263366)
 Celsius(4536.700544887301)
 Celsius(4244.499046332409)
 Celsius(2393.3256334193343)
 Celsius(3426.084192861852)
 Celsius(1833.3630732706556)
 Celsius(2861.567316507572)
 Celsius(4417.214425816825)
 Celsius(2071.4927028808884)
 Celsius(4235.330597356379)
 Celsius(1433.3856245813436)
 Celsius(4640.47517862231)
 ⋮
 Celsius(4048.323912553685)
 Celsius(1416.2272594519764)
 Celsius(2761.198738916759)
 Celsius(-152.33273534497403)
 Celsius(2400.2444327410235)
 Celsius(1161.0198134085203)
 Celsius(1675.2363151882887)
 Celsius(-78.17953834281818)
 Celsius(3519.794877030177)
 Celsius(412.7360842196546)
 Celsius(1910.8880274481216)
 Celsius(4287.95453542271)

### Is this fast?

In [37]:
@benchmark Celsius.($temperatures)

BenchmarkTools.Trial: 6331 samples with 1 evaluation.
 Range (min … max):  192.583 μs …   2.719 ms  ┊ GC (min … max):  0.00% … 68.96%
 Time  (median):     609.333 μs               ┊ GC (median):     0.00%
 Time  (mean ± σ):   787.350 μs ± 411.149 μs  ┊ GC (mean ± σ):  23.06% ± 23.99%

  ▁           ▃██▆▄▁                         ▁▂▂▁ ▁▂▁ ▁▂▂▂▂▁  ▁ ▂
  █▁▁▁▁▁▁▁▁▁▁▁████████▇▇▄▁▃▄▁▁▁▁▁▁▁▁▁▁▃▁▃▅▇▇███████████████████ █
  193 μs        Histogram: log(frequency) by time       1.95 ms <

 Memory estimate: 7.63 MiB, allocs estimate: 2.

### Yep... 1.5ms, 2 allocs, 8 MiB memory

## And we magically get all the other methods of the `rand()` function:

In [38]:
rand(Kelvin, 100, 200, 300)

100×200×300 Array{Kelvin, 3}:
[:, :, 1] =
 Kelvin(3102.21)  Kelvin(2440.81)  …  Kelvin(477.177)  Kelvin(69.445)
 Kelvin(3188.89)  Kelvin(731.13)      Kelvin(3844.25)  Kelvin(1781.22)
 Kelvin(2602.58)  Kelvin(3438.93)     Kelvin(3932.11)  Kelvin(4984.53)
 Kelvin(968.511)  Kelvin(1431.71)     Kelvin(2647.02)  Kelvin(4073.61)
 Kelvin(983.926)  Kelvin(1427.08)     Kelvin(1018.15)  Kelvin(2131.85)
 Kelvin(1197.94)  Kelvin(3253.41)  …  Kelvin(1959.28)  Kelvin(3938.65)
 Kelvin(1944.52)  Kelvin(1787.91)     Kelvin(1077.27)  Kelvin(3744.31)
 Kelvin(3478.48)  Kelvin(3904.12)     Kelvin(2419.2)   Kelvin(1078.34)
 Kelvin(2580.19)  Kelvin(1409.92)     Kelvin(2110.48)  Kelvin(4069.49)
 Kelvin(3999.33)  Kelvin(142.726)     Kelvin(433.969)  Kelvin(4466.24)
 Kelvin(4306.4)   Kelvin(1746.19)  …  Kelvin(1992.92)  Kelvin(2803.38)
 Kelvin(2795.91)  Kelvin(471.463)     Kelvin(3702.48)  Kelvin(639.113)
 Kelvin(4900.32)  Kelvin(3883.04)     Kelvin(916.808)  Kelvin(970.27)
 ⋮                                 ⋱ 